<a href="https://colab.research.google.com/github/jacksonkarel/etl-langchain/blob/main/langchain_karl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain
!pip install openai
!pip install google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import json
keys = json.load(open('keys.json'))
os.environ["OPENAI_API_KEY"] = keys['openai']
os.environ["SERPAPI_API_KEY"] = keys['serp']
import pandas as pd
import numpy as np

In [3]:
from langchain.agents import Tool
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain import OpenAI, SerpAPIWrapper, LLMChain
from langchain.utilities import PythonREPL
from langchain.agents import initialize_agent

In [5]:
python_repl = PythonREPL()

tools = [
    Tool(
        name="Python REPL",
        func=python_repl.run,
        description="A Python shell. Use this to execute python commands. Input should be a valid python command. Always print the output of the python command."
    ),
]

memory = ConversationBufferMemory(memory_key="chat_history")

llm=OpenAI(temperature=0)
agent_chain = initialize_agent(tools, llm, agent="conversational-react-description", verbose=True, memory=memory)

In [4]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent

In [23]:
tools = load_tools(["python_repl"], llm=llm)

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

input = """Execute the following steps:

1. Using python execution, load `dataset.csv`

2. Using python execution, iterate over the columns of the dataset printing the column name and datatype

3. Identify which columns are feature columns and which columns are label columns

4. Split the dataset into features and labels

5. Based on the data types, write python code for preprocessing and normalizing the features using sklearn

6. Add labels back to the processed dataframe

7. Save the processed data as `dataset_processed.csv`
"""

agent.run(input)



> Entering new AgentExecutor chain...
 I need to use the Python REPL to execute the steps
Action: Python REPL
Action Input: 
import pandas as pd

dataset = pd.read_csv('dataset.csv')

for col in dataset.columns:
    print(col, dataset[col].dtype)

Observation: alcohol float64
malic_acid float64
ash float64
alcalinity_of_ash float64
magnesium float64
total_phenols float64
flavanoids float64
nonflavanoid_phenols float64
proanthocyanins float64
color_intensity float64
hue float64
od280/od315_of_diluted_wines float64
proline float64
target int64

Thought: I need to identify which columns are feature columns and which columns are label columns
Action: Python REPL
Action Input: 
feature_columns = dataset.columns[:-1]
label_column = dataset.columns[-1]

Observation: 
Thought: I need to split the dataset into features and labels
Action: Python REPL
Action Input: 
features = dataset[feature_columns]
labels = dataset[label_column]

Observation: 
Thought: I need to preprocess and normalize the 

'The final answer is that the dataset has been preprocessed and normalized using sklearn, and saved as `dataset_processed.csv`.'

In [29]:
input = """Execute the following steps:

1. Using python execution, load `dataset_processed.csv`

2. Split the dataset into train and test sets

3. Select a machine learning model from the sklearn library

4. Fit the sklearn model to the training data

5. Use the trained model to predict on the test dataset

6. Evaluate the predictions using an appropriate metric. Print the result

7. Save the trained model to disk
"""

agent.run(input)



> Entering new AgentExecutor chain...
 I need to use the Python REPL to execute the steps
Action: Python REPL
Action Input: 
import pandas as pd
dataset = pd.read_csv('dataset_processed.csv')
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(dataset.drop('target', axis=1), dataset['target'], test_size=0.2, random_state=42)
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, predictions)
print(accuracy)
Observation: 1.0

Thought: I need to save the model to disk
Action: Python REPL
Action Input: 
import joblib
joblib.dump(model, 'model.joblib')
Observation: 
Thought: I now know the final answer
Final Answer: The model has been saved to disk as 'model.joblib'. The accuracy of the predictions is 1.0.

> Finished chain.


"The model has been saved to disk as 'model.joblib'. The accuracy of the predictions is 1.0."

In [30]:
tools = load_tools(["python_repl"], llm=llm)

agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)

input1 = """Execute the following steps:

1. Using python execution, load `dataset.csv`

3. Using python execution, split the dataset into train and test sets using a 80% random split

4. Save the datasets to `train_data.csv` and `test_data.csv`
"""

input2 = """Execute the following steps to train an sklearn model:

Using python execution, load `train_data.csv`

Using python execution, iterate over the columns of the dataset printing the column name and datatype

Identify which columns are feature columns and which columns are label columns

Split the dataset into features and labels

Determine if the problem is a classification or regression problem

Create an sklearn pipeline including preprocessing and modeling

Fit the pipeline to the dataset

Save the fitted pipeline to disk as `model.joblib`
"""

input3 = """Execute the following steps to run inference with a sklearn model:

1. Using python execution, load the sklearn model `model.joblib` from disk

2. Using python execution, load `test_data.csv`

3. Using python execution, iterate over the columns of the dataset printing the column name and datatype

4. Split the dataset into features and labels

5. Run prediction inference on the test data

6. Evaluate the predictions with an appropreate metric

7. Print the result
"""

In [23]:
agent.run(input1)



> Entering new AgentExecutor chain...
 I need to use the Python REPL to execute the steps
Action: Python REPL
Action Input: 
import pandas as pd
data = pd.read_csv('dataset.csv')
train_data, test_data = data.train_test_split(test_size=0.2, random_state=0)
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)
Observation: 'DataFrame' object has no attribute 'train_test_split'
Thought: I need to use the sklearn library
Action: Python REPL
Action Input: 
import pandas as pd
from sklearn.model_selection import train_test_split
data = pd.read_csv('dataset.csv')
train_data, test_data = train_test_split(data, test_size=0.2, random_state=0)
train_data.to_csv('train_data.csv', index=False)
test_data.to_csv('test_data.csv', index=False)
Observation: 
Thought: The datasets should now be saved
Final Answer: The datasets should now be saved to `train_data.csv` and `test_data.csv`.

> Finished chain.


'The datasets should now be saved to `train_data.csv` and `test_data.csv`.'

In [31]:
agent.run(input2)



> Entering new AgentExecutor chain...
 I need to use the Python REPL to execute the steps
Action: Python REPL
Action Input: 
import pandas as pd
train_data = pd.read_csv('train_data.csv')
for col in train_data.columns:
    print(col, train_data[col].dtype)
Observation: alcohol float64
malic_acid float64
ash float64
alcalinity_of_ash float64
magnesium float64
total_phenols float64
flavanoids float64
nonflavanoid_phenols float64
proanthocyanins float64
color_intensity float64
hue float64
od280/od315_of_diluted_wines float64
proline float64
target int64

Thought: I need to identify which columns are feature columns and which columns are label columns
Action: Python REPL
Action Input: 
features = train_data.drop('target', axis=1)
labels = train_data['target']
Observation: 
Thought: I need to split the dataset into features and labels
Action: Python REPL
Action Input: 
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)
Observation: name '

'The final answer is to execute the following steps to train an sklearn model: Using python execution, load `train_data.csv`, iterate over the columns of the dataset printing the column name and datatype, identify which columns are feature columns and which columns are label columns, split the dataset into features and labels, determine if the problem is a classification or regression problem, create an sklearn pipeline including preprocessing and modeling, fit the pipeline to the dataset, and save the fitted pipeline to disk as `model.joblib`.'

In [32]:
agent.run(input3)



> Entering new AgentExecutor chain...
 I need to use the Python REPL to execute the steps
Action: Python REPL
Action Input: 
import joblib
model = joblib.load('model.joblib')

import pandas as pd
test_data = pd.read_csv('test_data.csv')

for col in test_data.columns:
    print(col, test_data[col].dtype)

X = test_data.drop('label', axis=1)
y = test_data['label']

predictions = model.predict(X)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y, predictions)

print(accuracy)
Observation: "['label'] not found in axis"
Thought: I need to make sure I'm splitting the dataset correctly
Action: Python REPL
Action Input: 
X = test_data.drop('target', axis=1)
y = test_data['target']

predictions = model.predict(X)

from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y, predictions)

print(accuracy)
Observation: 0.9722222222222222

Thought: I now know the final answer
Final Answer: The accuracy of the model on the test data is 0.9722222222222222.

> Finish

'The accuracy of the model on the test data is 0.9722222222222222.'